<a href="https://colab.research.google.com/github/Thrishulgoud/Restaurant-ML-Project-task/blob/main/Restaurant_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Task-1 🍽️ Restaurant Recommendation System (Google Colab Project)
This notebook builds a **content-based recommendation system** to suggest restaurants based on user preferences.

In [ ]:
# Step 1: Upload Dataset
from google.colab import files
uploaded = files.upload()

import pandas as pd
# Read the uploaded file from the correct path and with the correct filename
data = pd.read_csv("/content/ Datasetml.csv")
print(data.head())
print(data.columns)
print(data.info())
print(data.isnull().sum())

Saving  Datasetml.csv to  Datasetml (1).csv
   Restaurant ID         Restaurant Name  Country Code              City  \
0        6317637        Le Petit Souffle           162       Makati City   
1        6304287        Izakaya Kikufuji           162       Makati City   
2        6300002  Heat - Edsa Shangri-La           162  Mandaluyong City   
3        6318506                    Ooma           162  Mandaluyong City   
4        6314302             Sambo Kojin           162  Mandaluyong City   

                                             Address  \
0  Third Floor, Century City Mall, Kalayaan Avenu...   
1  Little Tokyo, 2277 Chino Roces Avenue, Legaspi...   
2  Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...   
3  Third Floor, Mega Fashion Hall, SM Megamall, O...   
4  Third Floor, Mega Atrium, SM Megamall, Ortigas...   

                                     Locality  \
0   Century City Mall, Poblacion, Makati City   
1  Little Tokyo, Legaspi Village, Makati City   
2  Edsa Shangri

In [ ]:
# Step 2: Preprocessing
from sklearn.preprocessing import LabelEncoder

# Handle missing values
for col in data.columns:
    if data[col].dtype == 'object':
        data[col].fillna(data[col].mode()[0], inplace=True)
    else:
        data[col].fillna(data[col].mean(), inplace=True)

# Encode categorical columns
label_encoders = {}
for col in data.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

data.head()

/tmp/ipython-input-3988061134.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].mean(), inplace=True)
/tmp/ipython-input-3988061134.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,3748,162,73,8685,171,172,121.027535,14.565443,920,...,0,1,0,0,0,3,4.8,0,1,314
1,6304287,3172,162,73,6055,593,601,121.014101,14.553708,1111,...,0,1,0,0,0,3,4.5,0,1,591
2,6300002,2896,162,75,4684,308,314,121.056831,14.581404,1671,...,0,1,0,0,0,4,4.4,1,5,270
3,6318506,4707,162,75,8690,862,875,121.056475,14.585318,1126,...,0,0,0,0,0,4,4.9,0,1,365
4,6314302,5523,162,75,8689,862,875,121.057508,14.584450,1122,...,0,1,0,0,0,4,4.8,0,1,229


In [ ]:
# Step 3: Compute Similarity Matrix
from sklearn.metrics.pairwise import cosine_similarity

# Choose features for recommendation (adjust based on dataset)
features = ["Cuisines", "Price range", "Country Code"]
X = data[features]

# Compute similarity
similarity = cosine_similarity(X)
print("Similarity matrix computed with shape:", similarity.shape)

Similarity matrix computed with shape: (9551, 9551)


In [ ]:
# Step 4: Recommendation Function
def recommend_restaurants(user_pref, top_n=5):
    """
    user_pref: dict with keys like {"Cuisines": "Italian", "Price range": 3, "Country Code": 1}
    top_n: number of recommendations
    """
    # Encode user preferences
    user_vector = []
    for col in features:
        if col in label_encoders:  # categorical
            user_vector.append(label_encoders[col].transform([user_pref[col]])[0])
        else:
            user_vector.append(user_pref[col])

    user_vector = pd.DataFrame([user_vector], columns=features)

    # Compute similarity
    user_similarity = cosine_similarity(user_vector, X)[0]

    # Top recommendations
    indices = user_similarity.argsort()[-top_n:][::-1]
    return data.iloc[indices][["Restaurant Name", "Cuisines", "Price range", "Aggregate rating"]]

In [ ]:
# Step 5: Test the Recommendation System
user_pref = {"Cuisines": "Italian", "Price range": 3, "Country Code": 1}
recommendations = recommend_restaurants(user_pref, top_n=5)
print("Recommended Restaurants:")
print(recommendations)

Recommended Restaurants:
      Restaurant Name  Cuisines  Price range  Aggregate rating
9275             2383      1031            3               4.4
3219               37      1031            3               3.9
9140             6749      1031            3               3.6
9083             5340      1031            3               3.1
943              1156      1031            3               4.5
